In [ ]:
import numpy as np
import wandb
from tensorflow.keras.datasets import fashion_mnist

# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values
X_train = X_train / 255.0  # Scale between 0 and 1

# Class names in Fashion MNIST
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

# Sweep configuration
sweep_config = {
    "method": "random",
    "metric": {
        "name": "sample_image_quality",
        "goal": "maximize"
    },
    "parameters": {
        "num_hidden_layers": {"values": [1, 2, 3]},
        "num_neurons": {"values": [16, 32, 64]}
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="feedforward-nn")

# Function for each sweep run
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Find one image per class
        class_images = {}
        for image, label in zip(X_train, y_train):
            if label not in class_images:
                class_images[label] = image
            if len(class_images) == len(class_names):
                break

        # Log one image per class
        for label, image in sorted(class_images.items()):
            # Ensure the image has a channel dimension
            image = np.expand_dims(image, axis=-1)

            wandb.log({
                f"{class_names[label]}": wandb.Image(image, caption=class_names[label])
            })

        # Dummy metric for sweeps
        wandb.log({"sample_image_quality": np.random.rand()})

# Start the sweep
wandb.agent(sweep_id, train, count=5)


Create sweep with ID: bwg3azdf
Sweep URL: https://wandb.ai/bgorai005-iit-madras/feedforward-nn/sweeps/bwg3azdf


wandb: Agent Starting Run: 0v1ka65w with config:
wandb: 	num_hidden_layers: 1
wandb: 	num_neurons: 32


sample_image_quality,▁
sample_image_quality,0.50199


wandb: Agent Starting Run: m7hbzmon with config:
wandb: 	num_hidden_layers: 3
wandb: 	num_neurons: 32


sample_image_quality,▁
sample_image_quality,0.97007


...

sample_image_quality,▁
sample_image_quality,0.58199


wandb: Agent Starting Run: 6y3xpnqt with config:
wandb: 	num_hidden_layers: 2
wandb: 	num_neurons: 16


sample_image_quality,▁
sample_image_quality,0.73548


wandb: Agent Starting Run: qui2b45k with config:
wandb: 	num_hidden_layers: 1
wandb: 	num_neurons: 64


sample_image_quality,▁
sample_image_quality,0.53929


In [ ]:
# Flatten the images (28x28 -> 784)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


In [ ]:
# function for one hot encoding
def one_hot_encode(labels, num_classes):
    encoded =np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
       encoded[i,label]=1
    return encoded


In [ ]:
#apply one hot encoding in output column
y_train = one_hot_encode(y_train, 10)
y_test = one_hot_encode(y_test, 10)


In [ ]:
#class feedforward nueral network
class Feedforwardneuralnetwork:
  def __init__(self,input_size,hidden_layers,neurons_per_layer,output_size,learning_rate=0.01):
    self.layers=[]
    self.learning_rate=learning_rate
    previous_size=input_size
    for i in range(hidden_layers):
      self.layers.append(self.initialize_weights(previous_size,neurons_per_layer))
      previous_size=neurons_per_layer
    self.layers.append(self.initialize_weights(previous_size,output_size))

  def initialize_weights(self,input_size,output_size):
    weights=np.random.rand(input_size,output_size)*learning_rate
    biases =np.zeros((1,output_size))
    return (weights,biases)

  #def of sigmoid function
  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  #def of softmax function
  def softmax(self,x):
    exps = np.exp(x-np.max(x, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1,keepdims=True)

  #def of activation function
  def activation_function(x,active_fun):
    if active_fun=="sigmoid":
      return sigmoid(x)


  #def of derivative activation function
  def dervative_activation_function(x,active_fun):
    if active_fun=="sigmoid":
      return derivative_sigmoid(x)


  #for more flexible asa number of neurons and number of hidden layer
  def forward_propagation_flex(self,x,theta,active_fun,num_hidden_layer):
    a={}
    h={'h0':x}
    for i in range(1,num_hidden_layer+1):
      a["a"+str(i)]=np.dot(theta["W"+str(i)]),h["h"+str(i-1)]+theta['b'+str(i)]
      h["h"+str(i)]=activation_function(a["a"+str(i)],active_fun)
    #for output layer
    a["a"+str(num_hidden_layer+1)]=np.dot(theta['W'+str(num_hidden_layer+1)],h['h'+str(num_hidden_layer+1)])
    y_pred=softmax(a['a'+str(num_hidden_layer+1)])

    return a,h,y_pred



  #def of backprop
  def backprop(self,x,y_actual,active_fun,num_hidden_layers,theta,input_size):
    a,h,y_pred=forward_propagation_flex(x,theta,active_fun,num_hidden_layers)
    grad_theta ={}
    grad_layers ={}
    grad_layers['a'+str(num_hidden_layers+1)]=-1*(y_pred-y_actual)
    for i in range(num_hidden_layers+1,0,-1):
      grad_theta['W'+str(i)]=np.dot(grad_layers['a'+str(i)],h['h'+str(i-1).T])
      grad_theta['b'+str(i)]=np.sum(grad_layers['a'+str(i)],axis=0)
      grad_layers['h'+str(i-1)]=np.dot(theta['W'+str(i)].T,grad_layers['a'+str(i)])
      grad_layers['a'+str(i-1)]=grad_layers['h'+str(i-1)]*(derivative_activation_function(a['h'+str(i-1)],active_fun))
    return grad_theta

  def gradient_descent(self,x,y_actual,active_fun,num_hidden_layers,theta,input_size,epoch,learning_rate):
    for i in range(epoch):
      grad_theta=backprop(x,y_actual,active_fun,num_hidden_layers,theta,input_size)
      for j in range(1,num_hidden_layer+1):
        theta['W'+str(j)]=theta['W'+str(j)]-learning_rate*grad_theta['W'+str(j)]
        theta['b'+str(j)]=theta['b'+str(j)]-learning_rate*grad_theta['b'+str(j)]
    return theta


#derivative of function
def derivative_sigmoid(x):
  return x*(1-x)
#derivative of softmax functiom
def derivative_softmax(x):
  exps = np.exp(x-np.max(x, axis=1, keepdims=True))
  return exps/np








In [ ]:
import numpy as np



# de of forward propagation
def forward_propagation(x,n_hiddenlayer,n_classes):
  np.random.seed(42)

  n_samples,n_feature=x.shape


  #initialize the weight and biases
  w1=np.random.rand(n_feature,n_hiddenlayer)
  b1=np.random.rand(1,n_hiddenlayer)
  w2=np.random.rand(n_hiddenlayer,n_classes)
  b2=np.random.rand(1,n_classes)

  #forward pass
  z1=np.dot(x,w1)+b1
  A1=sigmoid(z1)

  #output layer
  z2=np.dot(A1,w2)+b2
  A2=softmax(z2)


  return A2



In [ ]:


#def of sigmoid function
def sigmoid(x):
  return 1/(1+np.exp(-x))

#def of softmax function
def softmax(x):
  exps = np.exp(x-np.max(x, axis=1, keepdims=True))
  return exps/np.sum(exps, axis=1,keepdims=True)





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

#load the fashion mnist-dataset
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

train_labels


array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
import numpy as np
import pandas as pd

# Load the Fashion-MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

# One-hot encode the labels
num_classes = 10
train_labels = np.eye(num_classes)[train_labels]
test_labels = np.eye(num_classes)[test_labels]

# Flatten the images
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

# Define the neural network
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_layers, neurons_per_layer, output_size, learning_rate=0.01):
        self.layers = []
        self.learning_rate = learning_rate
        previous_size = input_size
        for _ in range(hidden_layers):
            self.layers.append(self.initialize_weights(previous_size, neurons_per_layer))
            previous_size = neurons_per_layer
        self.layers.append(self.initialize_weights(previous_size, output_size))

    def initialize_weights(self, input_size, output_size):
        weights = np.random.randn(input_size, output_size) * 0.01
        biases = np.zeros((1, output_size))
        return (weights, biases)

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def forward(self, x):
        activations = [x]
        for weights, biases in self.layers[:-1]:
            z = np.dot(activations[-1], weights) + biases
            activations.append(self.relu(z))
        final_weights, final_biases = self.layers[-1]
        z = np.dot(activations[-1], final_weights) + final_biases
        output = self.softmax(z)
        activations.append(output)
        return activations

    def backward(self, activations, labels):
        grads = []
        m = labels.shape[0]
        output_error = activations[-1] - labels

        for i in range(len(self.layers) - 1, 0, -1):
            weights, biases = self.layers[i]
            dw = np.dot(activations[i - 1].T, output_error) / m
            db = np.sum(output_error, axis=0, keepdims=True) / m
            grads.append((dw, db))

            if i > 1:
                output_error = np.dot(output_error, weights.T) * self.relu_derivative(activations[i - 1])

        grads.reverse()
        return grads

    def update_weights(self, grads):
        for i, (dw, db) in enumerate(grads):
            self.layers[i] = (self.layers[i][0] - self.learning_rate * dw, self.layers[i][1] - self.learning_rate * db)

    def compute_loss(self, predictions, labels):
        m = labels.shape[0]
        loss = -np.sum(labels * np.log(predictions + 1e-8)) / m
        return loss

    def accuracy(self, predictions, labels):
        pred_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(labels, axis=1)
        return np.mean(pred_labels == true_labels)

    def train(self, x, y, epochs=10, batch_size=64):
        for epoch in range(epochs):
            for i in range(0, x.shape[0], batch_size):
                x_batch = x[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                activations = self.forward(x_batch)
                grads = self.backward(activations, y_batch)
                self.update_weights(grads)

            predictions = self.forward(x)[-1]
            loss = self.compute_loss(predictions, y)
            acc = self.accuracy(predictions, y)
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss:.4f} - Accuracy: {acc:.4f}")

# Build and train the model
input_size = train_images.shape[1]
output_size = num_classes
hidden_layers = 2
neurons_per_layer = 128
learning_rate = 0.01

nn = SimpleNeuralNetwork(input_size, hidden_layers, neurons_per_layer, output_size, learning_rate)

nn.train(train_images, train_labels, epochs=10, batch_size=64)

# Evaluate on test data
predictions = nn.forward(test_images)[-1]
test_loss = nn.compute_loss(predictions, test_labels)
test_acc = nn.accuracy(predictions, test_labels)

print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


ValueError: operands could not be broadcast together with shapes (128,128) (128,10) 

In [1]:
print("hello world")

hello world
